In [3]:
import pandas as pd
import numpy as np
import duckdb
import warnings
import os

import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

# email
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

import warnings
from datetime import datetime, timedelta
from haversine import haversine

pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

categories = ['PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA', 'TPM', 'STL', 'BLK', 'STL_BLK']
con = duckdb.connect(database=":memory:")

cwd = os.path.abspath(os.getcwd()).replace("\\", "/")
if cwd.startswith("C:/Users/Rodolfo/"):
    RUN_LOCATION = "local"
else:
    RUN_LOCATION = "cloud"
time_offset = {"local": 3, "cloud": -5}
now = str((datetime.now() + timedelta(hours=time_offset[RUN_LOCATION]) + timedelta(hours=-3)).date())
print(f"Today's date:", now)

Today's date: 2026-01-05


In [4]:
%run ./common_utils.ipynb

# Initial Functions

In [5]:
def email(model, error):
    
    # Email details
    sender_email = "rodolfoe7157@gmail.com"
    receiver_email = "rodolfoe7157@gmail.com"
    password = "cqgu bfey cnyx sfue"  # See note below

    subject = "NBA create_Predictions error"
    body = f"Model: {model}_model\nERROR: {error}"

    # Create message
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))

    # Connect to Gmail SMTP server and send
    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
        server.login(sender_email, password)
        server.send_message(msg)

    print("Email sent successfully!")

In [6]:
def create_base_df():
    df = pd.DataFrame()
    df2 = pd.DataFrame()
    df3 = pd.DataFrame()
    df4 = pd.DataFrame()
    for i in [2022, 2023, 2024, 2025]:
        df_actuals = pd.read_csv(f"../tables/{i}/parlay_stats.csv")
        df_actuals['Season'] = i
        df = pd.concat([df, df_actuals])

        df_schd = pd.read_csv(f"../tables/{i}/nba_schedule.csv")
        df_schd['Season'] = i
        df2 = pd.concat([df2, df_schd])

        df_gms = pd.read_csv(f"../tables/{i}/season_gamelogs.csv")
        df_gms['Season'] = i
        df3 = pd.concat([df3, df_gms])

        df_inj = pd.read_csv(f"../tables/{i}/injuries.csv")
        df_inj['Season'] = i
        df4 = pd.concat([df4, df_inj])

    df['Date'] = pd.to_datetime(df.Date)
    df2['Date'] = pd.to_datetime(df2.Date)
    df3['Date'] = pd.to_datetime(df3.Date)
    df3 = df3[~df3[['Date', 'Team', 'Player']].duplicated(keep='last')]
    df4['Date'] = pd.to_datetime(df4.Date)

    df3_temp = df3.rename(columns={"3PM": "TPM", "3PA": "TPA", "3P%": "TP%", "TRB": "REB"}).drop(['Pos', 'Opp'], axis=1)
    df3_temp['PR'] = df3_temp.PTS + df3_temp.REB 
    df3_temp['PA'] = df3_temp.PTS + df3_temp.AST
    df3_temp['RA'] = df3_temp.REB + df3_temp.AST
    df3_temp['PRA'] = df3_temp.PTS + df3_temp.REB + df3_temp.AST
    df3_temp['STL_BLK'] = df3_temp.STL + df3_temp.BLK
    df = df.merge(df3_temp, on=['Season', 'Date', 'Team', 'Player'], how='left')

    df_mtch = df2[['Season', 'Date', 'AwayABV', 'HomeABV', 'AwayPTS', 'HomePTS', 'AwayB2B', 'HomeB2B', 'is_OT', 'cup_gm', 'pstszn_gm']]
    df_mtch['Team_type'] = 'Away'
    df_mtch = df_mtch.rename(columns={"AwayABV": "Team", "HomeABV": "Opp", "AwayB2B": "B2B"})[['Season', 'Date', 'Team', 'AwayPTS', 'HomePTS', 'Opp', 'B2B', 'is_OT', 'cup_gm', 'pstszn_gm', 'Team_type']]
    df_mtch2 = df_mtch.copy().rename(columns={"Team": "Opp", "Opp": "Team", "HomeB2B": "B2B"})[['Season', 'Date', 'Team', 'AwayPTS', 'HomePTS', 'Opp', 'B2B', 'is_OT', 'cup_gm', 'pstszn_gm']]
    df_mtch2['Team_type'] = 'Home'
    df_mtch = pd.concat([df_mtch, df_mtch2])
    df_mtch = df_mtch[['Season', 'Date', 'Team', 'Team_type', 'AwayPTS', 'HomePTS', 'is_OT', 'cup_gm', 'pstszn_gm']]
    df_mtch = df_mtch.sort_values(["Team", "Date"])
    df_mtch['team_game_num'] = df_mtch.groupby(["Team", "Season"]).cumcount() + 1
    df_mtch['Spread'] = np.where(df_mtch.Team_type == 'Home', df_mtch.HomePTS - df_mtch.AwayPTS, df_mtch.AwayPTS - df_mtch.HomePTS)
    df_mtch['Total'] = df_mtch.AwayPTS + df_mtch.HomePTS
    df_mtch['is_Win'] = np.where(df_mtch.Spread > 0, 1, 0)
    df_mtch['Szn_Wins'] = df_mtch.groupby(['Season', 'Team'])['is_Win'].cumsum()
    df = df.drop(['Season', 'Team_type'], axis=1).merge(df_mtch, on=['Date', 'Team'])

    df = df.merge(df4[['Date', 'Team', 'Player', 'Status']], on=['Date', 'Team', 'Player'], how='left')
    df['Status'] = np.where((df.Active == 1) & (df.Status.isnull()), 'Available', df.Status)
    df['Status'] = np.where((df.Active == 0), 'Out', df.Status)
    df['Status'] = np.where((df.Status == 'Out') & (df.Active != 0), 'Available', df.Status)
    
    return df

# Minutes Projection Model

In [7]:
def setup_df_mins(con, df):
    
    df = df[['Season', 'Date', 'Team', 'Team_type', 'Opp', 'Player', 'Pos', 'B2B', 'MP',
             'Spread', 'Total', 'team_game_num', 'Szn_Wins', 'pstszn_gm', 'is_OT']]
    
    for col in ['MP']:
        for N in [1, 3, 5, 10]:
            df[f'{col}_L{N}_avg'] = (
                df.groupby(['Player', 'Season'])[col]
                  .rolling(window=N, min_periods=1)
                  .mean()
                  .shift(1)
                  .reset_index(level=[0, 1], drop=True)
            )

    games_last_7_days = df.sort_values(['Player', 'Season', 'Date']).groupby(['Player', 'Season']).rolling('7D', on='Date', closed='left')['MP'].count().reset_index().rename(columns={"MP": "gms_L7_days"})
    games_last_7_days = games_last_7_days.drop_duplicates(
        subset=['Player', 'Season', 'Date']
    )
    df = df.merge(games_last_7_days, on=['Player', 'Season', 'Date'])
    df['gms_L7_days'] = df.gms_L7_days.fillna(0).astype(int)
    
    df['prev_team_mins_pct'] = (df.groupby(['Player', 'Season'])['MP'].shift(1)) / 240
           
    df['reserve_td'] = (df.MP < 8).astype(int)
    df['bench_td']   = ((df.MP >= 8) & (df.MP <= 25)).astype(int)
    df['starter_td'] = (df.MP > 25).astype(int)
    role_counts = df.groupby(['Season', 'Player'])[['reserve_td', 'bench_td', 'starter_td']].sum()
    role_counts['most_common_role'] = role_counts[['reserve_td', 'bench_td', 'starter_td']].idxmax(axis=1)
    role_counts['reserve'] = (role_counts['most_common_role'] == 'reserve_td').astype(int)
    role_counts['bench']   = (role_counts['most_common_role'] == 'bench_td').astype(int)
    role_counts['starter'] = (role_counts['most_common_role'] == 'starter_td').astype(int)
    df = df.merge(role_counts[['reserve', 'bench', 'starter']], on=['Season', 'Player'], how='left')
    
    df['role'] = 0
    df['role'] = np.where(df.starter == 1, 1, df.role)
    df['role'] = np.where(df.bench == 1, 2, df.role)
    df['role'] = np.where(df.reserve == 1, 3, df.role)
    
    for N in [1, 3, 5]:
        for role in ['reserve_td', 'bench_td', 'starter_td']:
            df[f'{role}_last{N}'] = (
                df.sort_values(['Player', 'Date']).groupby('Player')[role]
                  .rolling(N, min_periods=1)
                  .sum()
                  .shift(1)
                  .reset_index(0, drop=True)
            )
        rec_role_cols = [f'{role}_last{N}' for role in ['reserve_td', 'bench_td', 'starter_td']]
        df[f'recent_most_common_role_L{N}'] = df[rec_role_cols].idxmax(axis=1)
        df[f'recent_role_L{N}'] = 0
        df[f'recent_role_L{N}'] = np.where(df[f'recent_most_common_role_L{N}'] == f'starter_td_last{N}', 1, df[f'recent_role_L{N}'])
        df[f'recent_role_L{N}'] = np.where(df[f'recent_most_common_role_L{N}'] == f'bench_td_last{N}', 2, df[f'recent_role_L{N}'])
        df[f'recent_role_L{N}'] = np.where(df[f'recent_most_common_role_L{N}'] == f'reserve_td_last{N}', 3, df[f'recent_role_L{N}'])      
        df = df.drop(f'recent_most_common_role_L{N}', axis=1)
        for role in ['reserve_td', 'bench_td', 'starter_td']:
            df = df.drop(f'{role}_last{N}', axis=1)
      
    df['missed_games'] = (
        df.groupby(['Player', 'Team', 'Season'])['team_game_num']      
          .diff()
          .sub(1)
          .fillna(0)
          .astype(int)
    )
    
    df['game_spread_type'] = 0
    df['game_spread_type'] = np.where(abs(df.Spread < 6), 1, df.game_spread_type) 
    df['game_spread_type'] = np.where((abs(df.Spread >= 6) & abs(df.Spread <= 14)), 2, df.game_spread_type) 
    df['game_spread_type'] = np.where(abs(df.Spread > 14), 3, df.game_spread_type) 
    
    df['missed_games_after'] = (
        df.groupby(['Player', 'Team', 'Season'])['team_game_num']
          .diff(-1)
          .abs()
          .sub(1)
          .fillna(0)
          .astype(int)
    )
    df['MP_diff_next_avg5'] = df.groupby('Player')['MP'].shift(-1) - df['MP_L5_avg']
    df['Injured'] = (
                        ((df.missed_games_after > 0) & (df.MP_diff_next_avg5 < -10) & (df.role == 1)) | 
                        ((df.missed_games_after > 0) & (df.MP_diff_next_avg5 < -5) & (df.role == 2))
                    ).astype(int)
    
    df['return_game'] = (df.groupby('Player')['Injured'].shift(1) == 1).astype(int)
    df['games_since_return'] = (df.groupby('Player')['return_game'].cumsum())
    df['games_since_return'] = (df.groupby(['Player', 'games_since_return']).cumcount())
    df['ramp_phase'] = 0
    df.loc[df.return_game == 1, 'ramp_phase'] = 1
    df.loc[df.games_since_return.isin([1, 2]), 'ramp_phase'] = 2
    df.loc[df.games_since_return >= 3, 'ramp_phase'] = 3
    df['starter_return'] = ((df.return_game == 1) & (df.role == 1)).astype(int)
    df['bench_return']   = ((df.return_game == 1) & (df.role == 2)).astype(int)
    
    # TEST WITH OTHER INTERVALS OUTSIDE L5
    df['MP_drop_pct_L5'] = (df.MP_L5_avg - df.MP) / df.MP_L5_avg
    df['Early_stop'] = (
        (df['MP'] < 5) |
        ((df.role == 1) & (df.MP_drop_pct_L5 >= 0.35)) |   # starters
        ((df.role == 2) & (df.MP_drop_pct_L5 >= 0.45)) |   # bench
        ((df.role == 3) & (df.MP_drop_pct_L5 >= 0.55))     # reserve
    ).astype(int)
    
    for N in [1, 3, 5]:
        df[f'Early_stop_L{N}'] = df.sort_values(['Player', 'Date'])['Early_stop'].shift(1).rolling(N).sum()
        df[f'Injured_L{N}'] = df.sort_values(['Player', 'Date'])['Injured'].shift(1).rolling(N).sum()
    
    # Location based features
    df["DaysLstGm"] = (df.groupby("Player")["Date"].diff().dt.days).fillna(0).astype(int)
    df['Location'] = df.apply(lambda r: r['Team'] if r['Team_type'] == 'Home' else r['Opp'], axis=1)
    df['PrevLocation'] = df.groupby('Player')['Location'].shift(1)
    df['same_arena'] = (df['PrevLocation'] == df['Location']).astype(int)

    df = df.drop(['Season', 'Team_type', 'reserve_td', 'reserve', 'bench_td', 'bench', 'starter_td', 'starter', 'MP_drop_pct_L5',
                  'PrevLocation', 'Location', 'Injured', 'missed_games_after', 'MP_diff_next_avg5', 'return_game'], axis=1)    
    
    return df

# Main Model

In [8]:
def setup_df_main(df, tgt_stat):
    
    # Stat dependent features 
    if tgt_stat == 'PTS':
        tgt_stat_cols = []
        df = df[['Season', 'Date', 'Team', 'Opp', 'Player', 'Pos', 'MP', 
         'PTS', 'TPM', 'FG', 'FGA', 'TPA', 'FT', 'FTA', 
         f'Off_{tgt_stat}', f'Off_L3_{tgt_stat}', f'Off_L5_{tgt_stat}', f'Off_L10_{tgt_stat}', f'Off_{tgt_stat}_Rk',
         f'Def_{tgt_stat}', f'Def_L3_{tgt_stat}', f'Def_L5_{tgt_stat}', f'Def_L10_{tgt_stat}', f'Def_{tgt_stat}_Rk',
         'Spread', 'Total', 'is_OT']]
        
        # Efficiency metrics
        df['three_rate_raw'] =  np.where(df.FGA > 0, df['TPA'] / df['FGA'], 0)
        df['ft_rate_raw']    =  np.where(df.FGA > 0, df['FTA'] / df['FGA'], 0)
        df['eFG_raw'] = (df['FG'] + 0.5 * df['TPM']) / df['FGA']
        df['TS_raw'] = df['PTS'] / (2 * (df['FGA'] + 0.44 * df['FTA']))    
        df['usage_proxy_raw'] =  np.where(df.MP > 0, (df['FGA'] + 0.44 * df['FTA']) / df['MP'], 0)
        
        for w in [3, 5, 10]:
            for metric in ['three_rate', 'ft_rate', 'eFG', 'TS', 'usage_proxy']:
                col = f"{metric}_L{w}"
                df[col] = (
                    df.groupby(['Player','Season'])[f'{metric}_raw']
                      .rolling(w, min_periods=1)
                      .mean()
                      .shift(1)
                      .reset_index(level=[0,1], drop=True)
                )
        for metric in ['three_rate', 'ft_rate', 'eFG', 'TS', 'usage_proxy']:
            col = f'{metric}_weighted'
            df[col] = (
                0.6 * df[f'{metric}_L3'] +
                0.3 * df[f'{metric}_L5'] +
                0.1 * df[f'{metric}_L10']
            )
            df = df.drop(f'{metric}_raw', axis=1)
        
    elif tgt_stat == 'PRA':
        tgt_stat_cols = ['PTS', 'REB', 'AST', 'TOV']
        df = df[['Season', 'Date', 'Team', 'Opp', 'Player', 'Pos', 'MP', 
         'PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA', 'TPM', 'STL', 'BLK', 'STL_BLK', 'TOV',
         'FG', 'FGA', 'TPA', 'FT', 'FTA', 
         f'Off_{tgt_stat}', f'Off_L3_{tgt_stat}', f'Off_L5_{tgt_stat}', f'Off_L10_{tgt_stat}', f'Off_{tgt_stat}_Rk',
         f'Def_{tgt_stat}', f'Def_L3_{tgt_stat}', f'Def_L5_{tgt_stat}', f'Def_L10_{tgt_stat}', f'Def_{tgt_stat}_Rk',
         'Spread', 'Total', 'is_OT']]
        
        df['usage_proxy_raw'] =  np.where(df.MP > 0, (df['FGA'] + 0.44 * df['FTA']) / df['MP'], 0)
        for w in [3, 5, 10]:
            df[f"usage_proxy_L{w}"] = (
                df.groupby(['Player','Season'])[f'usage_proxy_raw']
                  .rolling(w, min_periods=1)
                  .mean()
                  .shift(1)
                  .reset_index(level=[0,1], drop=True)
            )
        df['usage_proxy_weighted'] = (
            0.6 * df[f'usage_proxy_L3'] +
            0.3 * df[f'usage_proxy_L5'] +
            0.1 * df[f'usage_proxy_L10']
        )
        df = df.drop('usage_proxy_raw', axis=1)
        
        
    else:
        tgt_stat_cols = []
        df = df[['Season', 'Date', 'Team', 'Opp', 'Player', 'Pos', 'MP', 
         'PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA', 'TPM', 'STL', 'BLK', 'STL_BLK',
         'FG', 'FGA', 'TPA', 'FT', 'FTA', 
          f'Off_{tgt_stat}', f'Off_L3_{tgt_stat}', f'Off_L5_{tgt_stat}', f'Off_L10_{tgt_stat}', f'Off_{tgt_stat}_Rk',
          f'Def_{tgt_stat}', f'Def_L3_{tgt_stat}', f'Def_L5_{tgt_stat}', f'Def_L10_{tgt_stat}', f'Def_{tgt_stat}_Rk',
         'Spread', 'Total', 'is_OT']]

    
    # Create rolling + lag features    
    for col in ['MP', 'FGA', 'TPA', 'FTA'] + tgt_stat_cols:
        df[f'{col}_lst_gm'] = (
            df
            .groupby(['Player', 'Season'])[col]
            .shift(1)
        )
        for N in [3, 5, 10]:
            df[f'{col}_last_{N}_avg'] = (
                df.groupby(['Player', 'Season'])[col]
                  .rolling(window=N, min_periods=1)
                  .mean()
                  .shift(1)
                  .reset_index(level=[0, 1], drop=True)
            )
            df[f"{col}_last_{N}_std"] = (
                df.groupby(['Player', 'Season'])[col]
                  .shift(1)
                  .rolling(window=N, min_periods=1)
                  .std()
            )

    # Role identifiers features
    df['reserve_td'] = (df.MP < 8).astype(int)
    df['bench_td']   = ((df.MP >= 8) & (df.MP <= 25)).astype(int)
    df['starter_td'] = (df.MP > 25).astype(int)
    role_counts = df.groupby(['Season', 'Player'])[['reserve_td', 'bench_td', 'starter_td']].sum()
    role_counts['most_common_role'] = role_counts[['reserve_td', 'bench_td', 'starter_td']].idxmax(axis=1)
    role_counts['reserve'] = (role_counts['most_common_role'] == 'reserve_td').astype(int)
    role_counts['bench']   = (role_counts['most_common_role'] == 'bench_td').astype(int)
    role_counts['starter'] = (role_counts['most_common_role'] == 'starter_td').astype(int)
    df = df.merge(role_counts[['reserve', 'bench', 'starter']], on=['Season', 'Player'], how='left')
    df['role'] = 0
    df['role'] = np.where(df.starter == 1, 1, df.role)
    df['role'] = np.where(df.bench == 1, 2, df.role)
    df['role'] = np.where(df.reserve == 1, 3, df.role)
    
    for col in categories + ['Season', 'FG', 'FGA', 'FT', 'FTA', 'TPM', 'TPA', 
                             'reserve_td', 'reserve', 'bench_td', 'bench', 'starter_td', 'starter'] + tgt_stat_cols:
        if col == tgt_stat:
            continue
        if col in df.columns:
            df = df.drop(col, axis=1)
        
    return df

### Today's predictions

In [12]:
def generate_predictions(tgt_stat):
    team_encoder = LabelEncoder()
    player_encoder = LabelEncoder()
    team_type_encoder = LabelEncoder()
    position_encoder = LabelEncoder()
    status_encoder = LabelEncoder()
    
    df_pred = create_base_df()
    
    # Encode string cols
    team_encoder.fit(pd.concat([df_pred["Team"], df_pred["Opp"]], axis=0))
    df_pred["Team"] = team_encoder.transform(df_pred["Team"])
    df_pred["Opp"] = team_encoder.transform(df_pred["Opp"])
    df_pred["Player"] = player_encoder.fit_transform(df_pred["Player"])
    df_pred["Pos"] = position_encoder.fit_transform(df_pred["Pos"])
    df_pred['Team_type'] = team_type_encoder.fit_transform(df_pred['Team_type'])
    df_pred["Status"] = status_encoder.fit_transform(df_pred["Status"])
    
    mins_model = xgb.XGBRegressor()
    mins_model.load_model("../ML_models/mins_model.json")
    stat_model = xgb.XGBRegressor()
    stat_model.load_model(f"../ML_models/{tgt_stat}_model.json")
    
    df_lines = pd.read_csv(f"../tables/2025/parlay_lines.csv")
    df_lines['Date'] = pd.to_datetime(df_lines.Date)
    df_lines = df_lines[~(df_lines.Team.isnull())]

    # Predict Mins
    df_lines["Team"] = team_encoder.transform(df_lines["Team"])
    df_pred = df_pred.merge(df_lines[['Date', 'Team', 'Spread', 'Total']], on=['Date', 'Team'], how='left')
    df_pred = df_pred[~df_pred[['Date', 'Team', 'Player']].duplicated(keep='last')]
    df_pred['Spread_x'] = np.where(df_pred.Spread_x.isnull(), df_pred.Spread_y, df_pred.Spread_x)
    df_pred['Total_x'] = np.where(df_pred.Total_x.isnull(), df_pred.Total_y, df_pred.Total_x)
    df_pred = df_pred.rename(columns={"Spread_x": "Spread", "Total_x": "Total"}).drop(['Spread_y', 'Total_y'], axis=1)
    df_pred_mins = setup_df_mins(con, df_pred)
    df_pred_mins = df_pred_mins.drop(['Date', 'MP'], axis=1)
    df_pred['MP'] = mins_model.predict(df_pred_mins)

    # Predict Stat
    df_pred = setup_df_main(df_pred, tgt_stat)
    feature_cols = [col for col in df_pred.columns if col not in ['Date', tgt_stat]]
    df_pred = df_pred[df_pred.Date == now][feature_cols]
    df_pred[f"{tgt_stat}_proj"] = stat_model.predict(df_pred)

    df_pred['Team'] = team_encoder.inverse_transform(df_pred["Team"])
    df_lines['Team'] = team_encoder.inverse_transform(df_lines["Team"])
    df_pred['Opp'] = team_encoder.inverse_transform(df_pred["Opp"])
    df_pred['Player'] = player_encoder.inverse_transform(df_pred["Player"])
    df_pred['Pos'] = position_encoder.inverse_transform(df_pred["Pos"])

    df_lines = df_lines[df_lines.Date == now][['Team', 'Player', f'{tgt_stat}_line']]
    df_pred = df_pred.merge(df_lines, on=['Team', 'Player'])

    tds_picks = df_pred[~(df_pred[f'{tgt_stat}_line'].isnull())]\
                [['Team', 'Player', 'Pos', 'Opp', 'MP', 'MP_last_5_avg', f'{tgt_stat}_line', f'{tgt_stat}_proj']]
    tds_picks['Diff'] = abs((df_pred[f'{tgt_stat}_line'] - df_pred[f'{tgt_stat}_proj']))
    tds_picks['Diff2'] = abs((df_pred['MP'] - df_pred['MP_last_5_avg']))
    tds_picks = tds_picks.sort_values('Diff', ascending=False).drop(['Diff', 'Diff2'], axis=1)
    if tds_picks.shape[0] >= 50:
        print(tds_picks.shape[0], 'rows')
        for tm in tds_picks.Team.unique():
            display(tds_picks[tds_picks.Team == tm])
    else:
        display(tds_picks)
    tds_picks.insert(0, 'Date', pd.to_datetime(now))
    partition_save_df(tds_picks, f"../tables/2025/gmday_preds_{tgt_stat}.csv")

In [13]:
try: 
    generate_predictions('PTS')
except Exception as e:
    email('PTS', e)
    raise Exception(e)

94 rows


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PTS_line,PTS_proj
48,LAC,Kawhi Leonard,SF,GSW,37.382854,33.088436,28.5,36.117661
80,LAC,Ivica Zubac,C,GSW,25.624052,27.378575,12.5,16.412550
75,LAC,Nicolas Batum,PF,GSW,22.882723,24.190438,5.5,8.748205
90,LAC,Brook Lopez,C,GSW,21.586420,20.914415,6.5,9.222088
1,LAC,James Harden,PG,GSW,35.886635,32.700216,23.5,25.704287
70,LAC,John Collins,PF,GSW,27.455315,26.226956,11.5,12.829356
11,LAC,Kris Dunn,PG,GSW,29.676348,27.315850,7.5,7.082088


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PTS_line,PTS_proj
78,ATL,Kristaps Porzingis,C,TOR,24.386087,25.501392,13.5,20.646803
83,ATL,Onyeka Okongwu,C,TOR,33.884949,33.425439,15.5,20.018423
18,ATL,Nickeil Alexander-Walker,SG,TOR,36.810467,32.952562,20.5,24.728979
47,ATL,Jalen Johnson,SF,TOR,37.236099,35.115307,24.5,22.269531
56,ATL,Zaccharie Risacher,SF,TOR,26.127592,23.068871,9.5,7.787801
33,ATL,Dyson Daniels,SG,TOR,36.010021,34.312303,12.5,13.568230
15,ATL,Vit Krejci,PG,TOR,25.181181,24.901602,8.5,9.372118


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PTS_line,PTS_proj
50,CHO,Brandon Miller,SF,OKC,34.110985,33.853749,17.5,24.644911
19,CHO,Collin Sexton,SG,OKC,25.168566,23.449282,11.5,16.005505
53,CHO,Kon Knueppel,SF,OKC,31.778009,29.194036,15.5,19.010662
8,CHO,LaMelo Ball,PG,OKC,28.476482,28.420966,17.5,19.082407
65,CHO,Miles Bridges,PF,OKC,31.898188,29.365138,17.5,16.001192
89,CHO,Moussa Diabate,C,OKC,25.731979,26.601913,7.5,8.569631


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PTS_line,PTS_proj
38,CHI,Isaac Okoro,SG,BOS,26.562712,23.309611,8.5,14.635942
25,CHI,Coby White,SG,BOS,27.965988,27.535535,18.5,12.652491
77,CHI,Nikola Vucevic,C,BOS,33.411575,29.559726,17.5,21.458733
68,CHI,Matas Buzelis,PF,BOS,30.556528,26.358694,15.5,15.341595


...

,Team,Player,Pos,Opp,MP,MP_last_5_avg,PTS_line,PTS_proj
26,BOS,Derrick White,SG,CHI,36.220131,33.894851,18.5,21.514965
69,BOS,Sam Hauser,PF,CHI,21.373753,21.253914,7.5,9.890287
87,BOS,Luka Garza,C,CHI,19.385363,19.674841,8.5,10.534826
39,BOS,Jaylen Brown,SF,CHI,33.465321,31.143381,30.5,31.329815
81,BOS,Neemias Queta,C,CHI,26.343393,26.866297,10.5,10.993832
3,BOS,Payton Pritchard,PG,CHI,35.984173,33.326310,16.5,16.934774
32,BOS,Anfernee Simons,SG,CHI,26.570814,25.852776,12.5,12.871321


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PTS_line,PTS_proj
24,PHI,Quentin Grimes,SG,DEN,32.918354,32.116433,12.5,9.679493
20,PHI,VJ Edgecombe,SG,DEN,37.341988,35.073251,16.5,18.880861
76,PHI,Joel Embiid,C,DEN,33.390884,32.712575,24.5,26.334837
7,PHI,Tyrese Maxey,PG,DEN,37.863499,38.254156,27.5,29.305870
62,PHI,Paul George,PF,DEN,34.844147,33.183356,15.5,14.797766


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PTS_line,PTS_proj
71,HOU,Tari Eason,PF,PHO,27.165564,21.672292,14.5,11.955170
43,HOU,Kevin Durant,SF,PHO,36.849957,34.422539,27.5,25.123150
92,HOU,Steven Adams,C,PHO,20.751244,17.485040,7.5,5.345198
66,HOU,Jabari Smith Jr.,PF,PHO,35.781399,33.634413,15.5,14.119318
49,HOU,Amen Thompson,SF,PHO,35.387112,33.921542,18.5,17.944778
12,HOU,Reed Sheppard,PG,PHO,27.993750,25.717914,12.5,12.285341


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PTS_line,PTS_proj
22,GSW,Brandin Podziemski,SG,LAC,27.460646,28.489386,10.5,11.974551
27,GSW,Moses Moody,SG,LAC,23.458075,22.169621,8.5,7.736674
40,GSW,Jimmy Butler,SF,LAC,34.273724,33.178683,19.5,19.847631
9,GSW,Stephen Curry,PG,LAC,35.273331,33.970636,27.5,27.840538
74,GSW,Quinten Post,PF,LAC,20.444933,18.081041,7.5,7.829034
16,GSW,De'Anthony Melton,PG,LAC,23.151054,20.237901,8.5,8.621034
73,GSW,Draymond Green,PF,LAC,25.797251,25.111067,7.5,7.446731


../tables/2025/gmday_preds_PTS.csv saved!


In [14]:
try: 
    generate_predictions('PRA')
except Exception as e:
    email('PRA', e)
    raise Exception(e)

61 rows


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PRA_line,PRA_proj
48,LAC,Kawhi Leonard,SF,GSW,37.382854,33.088436,38.5,46.442932
1,LAC,James Harden,PG,GSW,35.886635,32.700216,36.5,33.122986
80,LAC,Ivica Zubac,C,GSW,25.624052,27.378575,24.5,25.890963
11,LAC,Kris Dunn,PG,GSW,29.676348,27.315850,13.5,12.377973


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PRA_line,PRA_proj
79,NYK,Karl-Anthony Towns,C,DET,30.658413,29.075297,35.5,42.524780
93,NYK,Mitchell Robinson,C,DET,20.028292,21.211502,12.5,15.959060
45,NYK,Mikal Bridges,SF,DET,38.061649,37.738794,24.5,23.204382
31,NYK,Miles McBride,SG,DET,30.786430,26.204542,17.5,18.131491
67,NYK,OG Anunoby,PF,DET,36.960728,35.636049,24.5,25.040377
4,NYK,Jalen Brunson,PG,DET,37.671844,37.972944,39.5,39.283386


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PRA_line,PRA_proj
83,ATL,Onyeka Okongwu,C,TOR,33.884949,33.425439,25.5,32.183319
33,ATL,Dyson Daniels,SG,TOR,36.010021,34.312303,25.5,28.133245
18,ATL,Nickeil Alexander-Walker,SG,TOR,36.810467,32.952562,27.5,28.235331
47,ATL,Jalen Johnson,SF,TOR,37.236099,35.115307,43.5,44.104870


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PRA_line,PRA_proj
53,CHO,Kon Knueppel,SF,OKC,31.778009,29.194036,24.5,30.818874
50,CHO,Brandon Miller,SF,OKC,34.110985,33.853749,26.5,31.751921
8,CHO,LaMelo Ball,PG,OKC,28.476482,28.420966,29.5,31.975313
65,CHO,Miles Bridges,PF,OKC,31.898188,29.365138,27.5,26.281214


...

,Team,Player,Pos,Opp,MP,MP_last_5_avg,PRA_line,PRA_proj
12,HOU,Reed Sheppard,PG,PHO,27.993750,25.717914,19.5,16.639927
49,HOU,Amen Thompson,SF,PHO,35.387112,33.921542,31.5,31.047337
43,HOU,Kevin Durant,SF,PHO,36.849957,34.422539,37.5,37.398071


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PRA_line,PRA_proj
40,GSW,Jimmy Butler,SF,LAC,34.273724,33.178683,29.5,32.273472
74,GSW,Quinten Post,PF,LAC,20.444933,18.081041,13.5,11.344278
22,GSW,Brandin Podziemski,SG,LAC,27.460646,28.489386,17.5,19.489893
27,GSW,Moses Moody,SG,LAC,23.458075,22.169621,13.5,12.545949
73,GSW,Draymond Green,PF,LAC,25.797251,25.111067,18.5,18.924097
9,GSW,Stephen Curry,PG,LAC,35.273331,33.970636,35.5,35.310356


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PRA_line,PRA_proj
46,TOR,RJ Barrett,SF,ATL,28.626137,29.941568,27.5,25.211926
63,TOR,Scottie Barnes,PF,ATL,36.413628,35.920688,34.5,36.059456
10,TOR,Immanuel Quickley,PG,ATL,33.699768,33.410746,26.5,25.333693
41,TOR,Brandon Ingram,SF,ATL,36.489079,35.344604,32.5,33.386330


,Team,Player,Pos,Opp,MP,MP_last_5_avg,PRA_line,PRA_proj
6,DEN,Jamal Murray,PG,PHI,37.436062,36.099972,39.5,41.650089


../tables/2025/gmday_preds_PRA.csv saved!
